# 🖥️ MITgcm tutorial

This tutorial will show how to run a 3D simulation with output from the MITgcm model. The key thing about MITgcm is that its output is on a C-grid, which means that the U and V velocity components are not defined at the same location as the particle positions. Parcels has built-in functionality to deal with C-grid data. See also the [Nemo tutorial](tutorial_nemo.ipynb) for another example of C-grid data.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import parcels

data_folder = parcels.download_example_dataset("MITgcm_example_data")
ds_fields = xr.open_dataset(data_folder / "mitgcm_UV_surface_zonally_reentrant.nc")

We can use a combination of `parcels.convert.mitgcm_to_sgrid` and `FieldSet.from_sgrid_conventions` to read in the data. See below for an example.

In [ ]:
ds_fset = parcels.convert.mitgcm_to_sgrid(
    fields={"U": ds_fields.UVEL, "V": ds_fields.VVEL}, coords=ds_fields
)
fieldset = parcels.FieldSet.from_sgrid_conventions(ds_fset)

We can then run a simulation on this zonally re-entrant dataset as follows:

In [ ]:
perc = np.arange(5, 100, 10)
lon_vals = np.percentile(fieldset.UV.grid.lon, perc)
lat_vals = np.percentile(fieldset.UV.grid.lat, perc)

X, Y = np.meshgrid(
    lon_vals,
    lat_vals,
)
Z = np.zeros_like(X)


def DeleteParticle(particles, fieldset):
    any_error = particles.state >= 50
    particles[any_error].state = parcels.StatusCode.Delete


pset = parcels.ParticleSet(fieldset=fieldset, lon=X, lat=Y, z=Z)

outputfile = parcels.ParticleFile(
    store="mitgcm_particles.zarr",
    outputdt=np.timedelta64(5000, "s"),
)

pset.execute(
    [parcels.kernels.AdvectionRK2, DeleteParticle],
    runtime=np.timedelta64(10, "D"),
    dt=np.timedelta64(1, "h"),
    output_file=outputfile,
)

And make a simple plot of the particle trajectories:

In [ ]:
ds = xr.open_zarr("mitgcm_particles.zarr")

plt.plot(ds.lon.T, ds.lat.T, ".-")
plt.show()